Login to Chameleon and download openrc.sh file from [here](https://testbed.expeca.proj.kth.se/project/api_access/openrc/). Upload it here next to this notebook and continue.

In the next cell, we setup the authentication method to be able to use Openstack clients.

In [ ]:
import os, re
from getpass import getpass

with open('nils-project-openrc.sh', 'r') as f:
    script_content = f.read()
    pattern = r'export\s+(\w+)\s*=\s*("[^"]+"|[^"\n]+)'
    matches = re.findall(pattern, script_content)

    for name, value in matches:
        os.environ[name] = value.strip('"')

# password read from file

Import packages

In [ ]:
import json
from loguru import logger
import chi.network, chi.container
from chi.expeca import reserve, list_reservations, unreserve_byid, get_container_status, wait_until_container_removed

Reserve the required equipment and resources

In [ ]:
# worker reservation
worker_lease = reserve(
    { "type":"device", "name":"worker-08", "duration": { "days":7, "hours":0 } }
)

worker_reservation_id = worker_lease["reservations"][0]["id"]

leaseslist = list_reservations(brief=True)
print(json.dumps(leaseslist,indent=4))

Create networks

In [ ]:
# Create edge-net for talker
edge_net = chi.network.create_network("edge-net")
edge_net_subnet = chi.network.create_subnet(
    "edge-net-subnet", 
    edge_net["id"], 
    "10.70.70.0/24", 
    gateway_ip="10.70.70.1", 
    enable_dhcp=False
)
logger.success("edge-net is created.")

# Create end-net for listener
end_net = chi.network.create_network("end-net")
end_net_subnet = chi.network.create_subnet(
    "end-net-subnet", 
    end_net["id"], 
    "10.42.3.0/24", 
    gateway_ip="10.42.3.1", 
    enable_dhcp=False
)
logger.success("end-net is created.")

Create routers

In [ ]:
end_net = chi.network.get_network("end-net")
edge_net = chi.network.get_network("edge-net")

In [ ]:
# Assuming router setup is already done and we just need to add subnets
router = chi.network.create_router("ros-router", "public")
chi.network.add_subnet_to_router(router["id"], edge_net_subnet["id"])
chi.network.add_subnet_to_router(router["id"], end_net_subnet["id"])
logger.success("Subnets added to the router.")

In [ ]:
# routeadd = chi.network.add_route_to_router(router["id"],"10.70.70.0/24","10.42.3.1")
# routeadd = chi.network.add_route_to_router(router["id"],"10.42.3.0/24","10.70.70.1")
# logger.success("the routes added to the router")

In [ ]:
public_net = chi.network.get_network("serverpublic")

# Deploy talker container on edge-net
talker_container = chi.container.create_container(
    name = "talker",
    image = "ros:humble-ros-base",
    reservation_id = worker_reservation_id,  # from previous reservations
        environment = {
        "DNS_IP":"8.8.8.8",
        "GATEWAY_IP":"130.237.11.97",
        "PASS":"expeca",
        "ROS_DOMAIN_ID": "30"
    },
    nets = [
        { "network" : public_net['id'] },
        { "network" : edge_net['id'] },
    ],
    labels = {
        "networks.1.interface":"ens1",
        "networks.1.ip":"130.237.11.114/27",
        "networks.1.gateway":"130.237.11.97",
        "networks.2.interface":"eno12419",
        "networks.2.ip":"10.70.70.3/24",
        "networks.2.routes":"172.16.0.0/16-10.70.70.1",
        "capabilities.privileged":"true",
    },
)
chi.container.wait_for_active("talker")
logger.success("Talker container deployed and active.")

In [ ]:
# Deploy listener container on end-net
listener_container = chi.container.create_container(
    name="listener",
    image="ros:humble-ros-base",
    reservation_id=worker_reservation_id,  # from previous reservations
    environment={"ROS_DOMAIN_ID": "30"},
    command="ros2 topic echo /chatter",
    nets=[{"network": end_net["id"]}]
)
chi.container.wait_for_active("listener")
logger.success("Listener container deployed and active.")
